# 품질검사전문기관 상세정보

In [1]:
from common import commonFunc as cf
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질검사전문기관 상세정보"
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질검사전문기관 상세정보/selectIoApiPmQtsc.csv'

In [2]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [3]:
JSONKEY = "items"
DUMMY = 0

In [4]:
# 페이지번호 설정
PAGEYN=1
if REQPARAM.count("pageNo") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
### 기본정보 설정 및 파라미터 설정 ###
PAGEYN

0

In [5]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/품질검사전문기관 목록/selectIoApiPmQtscList.csv",low_memory=False, encoding="ms949")

In [6]:
dfCombi = imsiDf.loc[:,["rgsseq"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
# dfCombiLen = dfCombi.shape[0]
# dfCombiLen

In [7]:
newParam = dfCombi.rgsseq.drop_duplicates().tolist()
len(newParam)

540

## 파라미터정보 불러오기

In [8]:
orgParam = []
try:
    # 기존 데이터 및 파라미터 정보 불러오기
    # 리스트로 불러오기로 변경 0425
    orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)[0]
## 파일이 없는경우
except Exception as e:
    orgParam = []
    print(e)

ptlcmnoList = list( set(newParam) - set(orgParam) )
# 모드: 0=종료 1=append 2=새로생성
mode = 2

# 업데이트 할 내용이 없으면 종료
if ptlcmnoList == []:
    mode = 0
    print("{} 정보 quit모드 {} ".format(DATANAME, mode))
    quit()
# orgParam == []
elif orgParam == []:
    mode = 2
    print("{} 정보 new 모드 {} ".format(DATANAME, mode))
# 이외에는 append 모드
else:
    mode = 1
    print("{} 정보 append 모드 {} ".format(DATANAME, mode))

# len(ptlcmnoList)

[Errno 2] No such file or directory: '../output/건설사업정보시스템/품질검사전문기관 상세정보/selectIoApiPmQtsc.pickle'
품질검사전문기관 상세정보 정보 new 모드 2 


In [9]:
resultDfMerged = pd.DataFrame()

In [11]:
numOfRows = 1000
resultDf = pd.DataFrame()
APICALL = 0 #추가

ptlcmnoListLen = len(ptlcmnoList)
pageList = [] #추가


for i in range(0,ptlcmnoListLen):
    BASEPARAM={"serviceKey":APIKEY[0], "rgsSeq": ptlcmnoList[i], "type":"json"}
    #print("BASEPARAM ",BASEPARAM)
    scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, APIKEY,APICALL,JSONKEY, DUMMY)
    resultDf = scrapyResult[0]
    pageList.append( scrapyResult[1] )
    APICALL = scrapyResult[2]
    if resultDf.empty: # 정상 데이터가 없는 경우
        print("No Data")
        break
    else:
        resultDfMerged = resultDfMerged.append(resultDf)


1 page scraping start apicall iter: 1 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'rgsSeq': 2005001, 'type': 'json'} rows: 15 completed
1 page scraping start apicall iter: 2 / used 0974320D-C689-4543-8464-10DE1E5505A9
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'rgsSeq': 2005002, 'type': 'json'} rows: 15 completed
1 page scraping start apicall iter: 3 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'rgsSeq': 2005003, 'type': 'json'} rows: 15 completed
1 page scraping start apicall iter: 4 / used 0974320D-C689-4543-8464-10DE1E5505A9
0 no pageNo
dataframe품질검사전문기관 상세정보, param:{'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'rgsSeq': 2005004, 'type': 'json'} rows: 15 completed
1 page scraping start apicall iter: 5 / used 49099D1D-E565-48E1-8A6F

KeyboardInterrupt: 

In [12]:
resultDfMerged = resultDfMerged.drop_duplicates()
resultDfMerged

,rgsseq,tpincts,perno,bsnrngcts,rgsdt,mtlnm,tpinno,rprnnm,tlno,zc,adr,fxno,lbrtare,orcd,orcdnm
0,2005001,"토목(골재,레미콘,아스콘,철강재)",익산2008-11,None,20050202,(주)한국건설시험연구원,None,최현기,061-375-0488,519803,전라남도 화순군 노대길 25,061-375-0412,156.0,A4000,None
0,2005002,"토목(골재,레미콘,아스콘,철강재,정재하,동재하)",익산2008-45,None,20050524,(유)대한건설품질시험원,None,백상헌,063-862-9991,570802,전북 익산시 함열읍 익산대로 1424-5,063-862-9995,152.5,A4000,None
0,2005003,특수,2013-01,섬유,20050721,(재)한국의류시험연구원부산지원,None,허재호,051-920-2701,612020,부산광역시 해운대구 센텀중앙로 48 610호,051-920-2710,240.0,B2000,None
0,2005004,특수,2010-17,섬유,20051107,(재)FITI시험연구원 부산지원,None,윤성광,051-463-5477,601838,부산광역시 동구 중앙대로248번길 14,051-462-8803,247.0,B2000,None
0,2005005,특수,"제2013-24, 25호","골재,레디믹스트콘크리트",20051118,고려엔지니어링(주),None,지봉복,031-285-8200,449831,경기도 용인시 처인구 이동면 백옥대로 260번길 39 (시미리 102-4번지),031-287-6236,109.0,A1000,None
0,2003001,"토목(골재,레미콘,아스콘,철강재)",익산2009-8,None,20030422,(주)한국건설산업시험연구원,410-81-79265,이민순,062-946-0186,502837,광주광역시 서구 화정로179번길 43,062-946-0188,215.5,A4000,None
0,2003002,"토목(골재,레미콘,아스콘,철강재,정재하,동재하)",익산2008-10,None,20030623,한국품질시험원(주),None,김생순,062-972-6701,502861,광주광역시 서구 유덕로27번길 13,062-972-6704,154.0,A4000,None
0,2003003,토목,2013-10,"골재,레디믹스트콘크리트,아스팔트콘크리트,철강재,말뚝재하(정재하시험,동재하시험)",20030811,건설기술원(주),None,김경영,054-971-0114,718912,경상북도 칠곡군 천평2길 29 한국품질시험연구소,054-971-0116,232.8,C4000,None


In [15]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME,mode)

품질검사전문기관 상세정보 save compled


In [16]:
resultDfMerged.shape

(540, 15)

In [13]:
# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
ptlcmnoList = orgParam + ptlcmnoList

paramList = [ptlcmnoList,pageList]
# 최종 파라미터 저장
cf.saveparam(paramList, SITENAME,DATANAME,SERVICENAME)